In [231]:

from astropy import units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import SphericalRepresentation

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.twobody.sampling import EpochsArray
from poliastro.twobody.propagation import CowellPropagator
from poliastro.util import Time, time_range

from haversine import haversine

import numpy as np

print('\nREVISIT TIME FOR A SPECIFIC LOCATION ON EARTH\n')
# initial orbital elements
alt  = 380 * u.km
a    = (Earth.R).to(u.km) + alt
ecc  = 0.001 * u.one
inc  = 96.9172647751491 * u.deg
raan = 0 * u.deg
argp = 0 * u.deg
nu   = 0 * u.deg 

start_date = Time("2023-01-01 00:00:00.000", scale = "utc")   # epoch

# sensor parameters
sw = 50*0.5 # [km]

# 25 km --> 0.22483 deg of latitude

target = (40.8518, 14.2681)   # Napoli latitude/longitude [deg]

# Definition of the initial orbit (poliastro)
in_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, start_date)

# Spacecraft data
C_D = 2.2  * u.one
A   = 0.01 * u.m**2
m   = 2.5  * u.kg


# Keplerian two-body propagation (poliastro)
time_frame = 7 * u.day   #float(input('Time frame [days]: '))
time_step  = 1 * u.s     #float(input('Time step [sec]: '))

number = int(time_frame.to_value(u.s) / time_step.value)
tofs = TimeDelta(np.linspace(0, time_frame, num=number))

# EarthLocation.to_geocentric()
# EarthLocation.to_geodetic('WGS84')


ephems = in_orbit.to_ephem(EpochsArray(start_date + tofs, CowellPropagator(rtol=1e-5)))
state, _ = ephems.rv()
rvec = state.value

access_time = []
revisit_time = []
# dist_list = []
# pos_list = []
i = 0

for idx in range(len(rvec)):
    
    curr_orbit = Orbit.from_ephem(Earth, ephems, ephems.epochs[idx])
    lat = (curr_orbit.represent_as(SphericalRepresentation).lat).to_value(u.deg)
    lon = (curr_orbit.represent_as(SphericalRepresentation).lon).to_value(u.deg)
    
    if lon > 180:
        lon = lon - 360

    pos = (lat, lon)
    dist = haversine(pos, target)

    if  dist < sw:
        access_time.append(ephems.epochs[idx])
    
        if (len(access_time)>1) and (((access_time[i].jd - access_time[i-1].jd)) > ((time_step).to_value(u.day))):
            revisit_time.append(access_time[i].jd - access_time[i-1].jd)

        i += 1


REVISIT TIME FOR A SPECIFIC LOCATION ON EARTH



KeyboardInterrupt: 